In [3]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import celerite as ce
import emcee as mc
import QPP_Funcs as qpp
import corner
from celerite.modeling import Model
from scipy.optimize import minimize, curve_fit

%matplotlib qt

i=0
plots = []
corners = []
datadir = "/Users/chris/Documents/QPP/SolarFlareGPs/data/daniela_goes_data/"

#for burstid in range(1520110128,1520170715):
while (i<1):
    burstid = 1520110128 #np.random.randint(1520110128,1520170715)
    try:
        time, flux = qpp.load_data(datadir,burstid)
    except(IOError):
        pass
    else:
        i+=1
        tt,ft = qpp.trim_data(time,flux)
        ft = ft*1e10  #cheating conversions!!!
        ft_err = np.sqrt(ft)
        plt.figure()
        print("Lightcurve " + str(i) + ", burstid: " + str(burstid))
        print("Initializing...")
        
        #initialization block
        A_guess, t1_guess, t2_guess = qpp.initguess(tt,ft)
        model = qpp.CTSModel_prior(log_A = np.log(A_guess), log_tau1 = np.log(t1_guess), log_tau2 = np.log(t2_guess))
        kernel1 = qpp.SHOTerm_Prior(log_S0 = 20, log_Q = -1, log_omega0 = -1) #write guesser for kernel parameters
        kernel2 = qpp.RealTerm_Prior(log_a=0., log_c=0.) #write guesser for kernel parameters
        kernel = kernel2 + kernel1
        gp = ce.GP(kernel, mean=model, fit_mean=True)
        gp.compute(tt, ft_err)
        print("Optimizing...")
        
        #optimization
        soln = qpp.optimize_gp(gp, ft)
        figopt = qpp.plot_gp(tt, ft, ft_err, gp, model, soln=soln)
    
        
        #sampling
        sampler = qpp.sample_gp(soln.x, ft, gp, burnin=1500, nsteps = 2000)
        chain = sampler.chain
        print("Plotting...")
        figsam = qpp.plot_chain(chain)
        
        figoptsam = qpp.plot_gp(tt, ft, ft_err, gp, model, chain=chain)
        
        figcorner, maxparams = qpp.plot_corner(chain, labels = gp.get_parameter_names())
        print maxparams
        plots.append(figoptsam)
        corners.append(figcorner)
plt.show()

Lightcurve 1, burstid: 1520110128
Initializing...
Optimizing...
Initial log-likelihood: -27078.3403895
Final log-likelihood: nan
Optimized log-parameters: [  0.           0.          20.          -1.          -1.          11.99069881
   7.21864007   7.91178725]
Burning in...
Sampling...
Done!
Plotting...


[ 18.80001714 -19.69935982  21.50131496   0.41230411  -3.98578729
  15.24645802   8.56109333   4.96689966]
